<h1> Bibliotecas </h1>

In [1]:
import numpy as np
import pandas as pd
import warnings                     #    É uma bibliote para ignorar os warnings do ambiente
import math
# Ignorando warnings desncessários
warnings.filterwarnings('ignore')
from pymongo import MongoClient
import pymongo
#AWS functions and features
import boto3
import base64
from botocore.exceptions import ClientError
import json

<h1> Mongo Database </h1>

In [2]:
client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId='Mongo')#coloco o nome do secret manager exatamente como foi escrito
secretMongo = json.loads(response["SecretString"])

In [3]:
decode_responses=True
mongolink = secretMongo["mongodb"]
client = pymongo.MongoClient(mongolink)
db1 = client.store

<h3> Ajustando campos </h3>

In [4]:
collection = db1["maquina"]
#collection = db1['winv']
data = pd.DataFrame(list(collection.find()))
data.drop(['_id'], axis=1, inplace=True)# removendo campos

In [5]:
#Comentei as linhas de drop pq ja foram realizadas.

In [6]:
data.drop(['Data'], axis=1, inplace=True)# removendo campos

In [7]:
data.drop(['Pontos'], axis=1, inplace=True)# removendo campos

In [8]:
data.drop(['Confirma'], axis=1, inplace=True)# removendo campos

In [9]:
data.drop(['MM09'], axis=1, inplace=True)# removendo campos

In [10]:
data.drop(['BearBull'], axis=1, inplace=True)# removendo campos

In [11]:
data.drop(['Tick_volume'], axis=1, inplace=True)# removendo campos

In [12]:
data.drop(['Volume'], axis=1, inplace=True)# removendo campos

In [13]:
data['target'] = data['Close'][1:len(data)].reset_index(drop=True)

In [18]:
pd.options.display.max_rows = 120
data.tail(3)

,Open,High,Low,Close,target
110,111035,111040,110830,110850,0.0
111,110845,110870,110605,110625,1.0
112,110630,110740,110600,110700,0.0


In [15]:
data.loc[data['target'] > data['Close'], 'target'] = 1 # Se 1 Close é maior do o target.

In [16]:
data.loc[data['target'] != 1, 'target'] = 0 # Se 0 Close é menor do o target.

In [17]:
data = data.drop(data[-1::].index, axis = 0) # a ultima linnha foi removida pois está ativa.
data.tail(2) 

,Open,High,Low,Close,target
111,110845,110870,110605,110625,1.0
112,110630,110740,110600,110700,0.0


In [ ]:
#data.loc[data['MM09'] > data['Close'], 'MM09'] = 1

In [ ]:
#data.loc[data['MM09'] != 1, 'MM09'] = 0

In [ ]:
#data.loc[data['Volume'] > 200000, 'Volume'] = 1

In [ ]:
#data.loc[data['Volume'] != 1, 'Volume'] = 0

In [ ]:
#data.loc[data['Tick_volume'] > 90000, 'Tick_volume'] = 1

In [ ]:
#data.loc[data['Tick_volume'] != 1, 'Tick_volume'] = 0

In [ ]:
#from sklearn.preprocessing import LabelEncoder

In [ ]:
#data['BearBull'] = LabelEncoder().fit_transform(data['BearBull'])

In [ ]:
#data = data.drop(data[-1::].index, axis = 0) #usei essa linha para validar a rede

In [19]:
data.shape

(113, 5)

In [ ]:
#data["BearBull"] = pd.to_numeric(data["BearBull"])

In [20]:
data.dtypes

Open        int64
High        int64
Low         int64
Close       int64
target    float64
dtype: object

**Rede NAIVE BAYES : Classificação**

**Alvos e Previsores**

In [21]:
previsores = data.iloc[:, 0:4].values

In [22]:
previsores

array([[113655, 114120, 112990, 113430],
       [113440, 113790, 113320, 113725],
       [113730, 113785, 113255, 113670],
       [113670, 113835, 113590, 113745],
       [113745, 113825, 113425, 113650],
       [113645, 113885, 113510, 113590],
       [113595, 113835, 113540, 113815],
       [113815, 113895, 113705, 113880],
       [113880, 113955, 113740, 113880],
       [113880, 113990, 113660, 113860],
       [113860, 114085, 113815, 113965],
       [113960, 114055, 113820, 114040],
       [114045, 114250, 113700, 114120],
       [114125, 114320, 113920, 114220],
       [114215, 114245, 113715, 113935],
       [113935, 113945, 113495, 113620],
       [113615, 114035, 113610, 113920],
       [113925, 114050, 113655, 113895],
       [113895, 114050, 113500, 113690],
       [113610, 113925, 113360, 113895],
       [113890, 113960, 113665, 113715],
       [113715, 113775, 113200, 113215],
       [113220, 113370, 112855, 112870],
       [112870, 113215, 112815, 113030],
       [113025, 

In [23]:
previsores.shape

(113, 4)

In [24]:
alvo = data.iloc[:,4].values

In [25]:
alvo

array([1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0.])

In [26]:
alvo.shape

(113,)

<h2> TREINO </h2>

In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [29]:
previsores_esc

array([[ 1.51202287,  1.77332458,  1.11352082,  1.3359658 ],
       [ 1.31706681,  1.48112683,  1.42616014,  1.60439097],
       [ 1.5800308 ,  1.47669959,  1.36457967,  1.5543456 ],
       [ 1.52562446,  1.52097198,  1.68195595,  1.62258929],
       [ 1.59363239,  1.5121175 ,  1.52563629,  1.53614728],
       [ 1.50295515,  1.56524436,  1.6061646 ,  1.48155233],
       [ 1.45761653,  1.52097198,  1.63458635,  1.6862834 ],
       [ 1.65710645,  1.57409884,  1.79090601,  1.74542792],
       [ 1.71604666,  1.6272257 ,  1.82406473,  1.74542792],
       [ 1.71604666,  1.65821638,  1.74827338,  1.72722961],
       [ 1.69791121,  1.74233391,  1.89511912,  1.82277077],
       [ 1.78858845,  1.71577048,  1.89985608,  1.89101446],
       [ 1.8656641 ,  1.88843279,  1.78616905,  1.96380772],
       [ 1.93820589,  1.95041413,  1.99459527,  2.05479931],
       [ 2.0198154 ,  1.88400555,  1.80037993,  1.79547329],
       [ 1.76591914,  1.61837123,  1.59195372,  1.50884981],
       [ 1.47575198,  1.

In [30]:
previsoresdf = pd.DataFrame(previsores_esc)

In [ ]:
previsoresdf

**Base Treino e Teste**

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
x_treino, x_teste, y_treino, y_teste = train_test_split(previsores_esc, alvo, test_size = 0.3, random_state = 0)

In [60]:
x_treino.shape

(79, 4)

In [61]:
x_teste.shape

(34, 4)

In [62]:
y_treino.shape

(79,)

In [63]:
y_teste.shape

(34,)

<h2> Criação do Algoritimo</h2>

In [64]:
from sklearn.naive_bayes import GaussianNB

In [65]:
naive = GaussianNB()

In [66]:
naive.fit(x_treino, y_treino)

GaussianNB()

In [67]:
previsoes = naive.predict(x_teste)

In [68]:
previsoes

array([0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0.])

In [69]:
y_teste

array([1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1.])

In [70]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [71]:
print('Acurácia: %.2f%%' % (accuracy_score(y_teste, previsoes)*100))

Acurácia: 38.24%


In [72]:
confusion_matrix(y_teste, previsoes)

array([[ 9,  9],
       [12,  4]], dtype=int64)

In [73]:
print(classification_report(y_teste, previsoes))

              precision    recall  f1-score   support

         0.0       0.43      0.50      0.46        18
         1.0       0.31      0.25      0.28        16

    accuracy                           0.38        34
   macro avg       0.37      0.38      0.37        34
weighted avg       0.37      0.38      0.37        34



**Analise de dados de treino**

In [74]:
previsoes_treino = naive.predict(x_treino)

In [75]:
previsoes_treino

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.])

In [76]:
accuracy_score(y_treino, previsoes_treino)

0.5189873417721519

In [77]:
confusion_matrix(y_treino, previsoes_treino)

array([[32, 11],
       [27,  9]], dtype=int64)

**Validação Cruzada**

In [78]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [79]:
kfold = KFold(n_splits = 30, shuffle=True, random_state = 5)

In [80]:
cruzada = GaussianNB()

In [81]:
resultado = cross_val_score(cruzada, previsores_esc, alvo, cv = kfold)

In [82]:
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100))

Acurácia Média: 49.17%


<h1>Criando Modelo</h1>

In [83]:
from sklearn.naive_bayes import GaussianNB

In [84]:
modelo = GaussianNB()

In [85]:
modelo.fit(previsores, alvo)

GaussianNB()

In [ ]:
#data[-2::] #Aqui fui para as duas ultimas linhas do dataframe já com as alterações

In [86]:
dependente_new = data.iloc[-2:-1, 0:4].values
dependente_new = dependente_new.reshape( 1, -1)
dependente_new

array([[110845, 110870, 110605, 110625]], dtype=int64)

In [87]:
print(modelo.predict(dependente_new))

[1.]


In [88]:
print('Acurácia: %.2f%%' % (accuracy_score(y_teste, previsoes)*100))

Acurácia: 38.24%


In [89]:
data.tail(5)

,Open,High,Low,Close,target
108,111055,111095,110980,111015,1.0
109,111020,111150,111000,111035,0.0
110,111035,111040,110830,110850,0.0
111,110845,110870,110605,110625,1.0
112,110630,110740,110600,110700,0.0
